# Evolver Loop 1 Analysis

## Goals:
1. Analyze N=1 theoretical optimal (is current solution optimal?)
2. Ensemble all pre-optimized submissions to find best per-N
3. Calculate potential improvement from ensemble

In [1]:
import pandas as pd
import numpy as np
import math
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
import glob
import os

getcontext().prec = 30

# Tree polygon vertices
TREE_VERTICES = [
    (0.0, 0.8),      # Tip
    (0.125, 0.5),    # Right top tier
    (0.0625, 0.5),
    (0.2, 0.25),     # Right mid tier
    (0.1, 0.25),
    (0.35, 0.0),     # Right base
    (0.075, 0.0),    # Right trunk
    (0.075, -0.2),
    (-0.075, -0.2),  # Left trunk
    (-0.075, 0.0),
    (-0.35, 0.0),    # Left base
    (-0.1, 0.25),    # Left mid tier
    (-0.2, 0.25),
    (-0.0625, 0.5),  # Left top tier
    (-0.125, 0.5),
]

print(f"Tree has {len(TREE_VERTICES)} vertices")

Tree has 15 vertices


In [2]:
# Analyze N=1 theoretical optimal
# For a single tree, the optimal bounding box is achieved at some rotation angle

def get_bbox_at_angle(angle_deg):
    """Get bounding box side length for a single tree at given angle"""
    poly = Polygon(TREE_VERTICES)
    rotated = affinity.rotate(poly, angle_deg, origin=(0, 0))
    minx, miny, maxx, maxy = rotated.bounds
    return max(maxx - minx, maxy - miny)

# Search for optimal angle
angles = np.linspace(0, 90, 901)  # 0.1 degree resolution
bbox_sides = [get_bbox_at_angle(a) for a in angles]

optimal_idx = np.argmin(bbox_sides)
optimal_angle = angles[optimal_idx]
optimal_side = bbox_sides[optimal_idx]

print(f"Optimal angle for N=1: {optimal_angle:.1f} degrees")
print(f"Optimal side length: {optimal_side:.6f}")
print(f"Optimal N=1 score contribution: {optimal_side**2 / 1:.6f}")

# Current baseline N=1 score
current_n1_side = 0.813  # from baseline analysis
current_n1_score = current_n1_side**2 / 1
print(f"\nCurrent N=1 side: {current_n1_side}")
print(f"Current N=1 score: {current_n1_score:.6f}")
print(f"Potential improvement: {current_n1_score - optimal_side**2:.6f}")

Optimal angle for N=1: 45.0 degrees
Optimal side length: 0.813173
Optimal N=1 score contribution: 0.661250

Current N=1 side: 0.813
Current N=1 score: 0.660969
Potential improvement: -0.000281


In [3]:
# Fine-tune around optimal angle
fine_angles = np.linspace(optimal_angle - 1, optimal_angle + 1, 2001)
fine_sides = [get_bbox_at_angle(a) for a in fine_angles]

fine_optimal_idx = np.argmin(fine_sides)
fine_optimal_angle = fine_angles[fine_optimal_idx]
fine_optimal_side = fine_sides[fine_optimal_idx]

print(f"Fine-tuned optimal angle: {fine_optimal_angle:.4f} degrees")
print(f"Fine-tuned optimal side: {fine_optimal_side:.8f}")
print(f"Fine-tuned N=1 score: {fine_optimal_side**2:.8f}")

Fine-tuned optimal angle: 45.0000 degrees
Fine-tuned optimal side: 0.81317280
Fine-tuned N=1 score: 0.66125000


In [4]:
# Load all pre-optimized submissions and find best per-N
from numba import njit

@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x,y

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def strip(a):
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

tx, ty = make_polygon_template()
print("Scoring functions ready")

Scoring functions ready


In [5]:
# Find all CSV files in snapshots
csv_files = glob.glob('/home/nonroot/snapshots/santa-2025/**/*.csv', recursive=True)
print(f"Found {len(csv_files)} CSV files")

# Filter out sample_submission.csv
csv_files = [f for f in csv_files if 'sample_submission' not in f]
print(f"After filtering: {len(csv_files)} CSV files")

Found 960 CSV files
After filtering: 898 CSV files


In [6]:
# Load and score all submissions
from tqdm import tqdm

best = {n: {'score': 1e300, 'data': None, 'src': None} for n in range(1, 201)}

for fp in tqdm(csv_files, desc='Scanning CSVs'):
    try:
        df = pd.read_csv(fp)
    except Exception:
        continue
    
    if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
        continue
    
    df = df.copy()
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    
    for n, g in df.groupby('N'):
        if n < 1 or n > 200:
            continue
        if len(g) != n:
            continue
        
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        ds = strip(g['deg'].to_numpy())
        sc = score_group(xs, ys, ds, tx, ty)
        
        if sc < best[n]['score']:
            best[n]['score'] = float(sc)
            best[n]['data'] = g.drop(columns=['N']).copy()
            best[n]['src'] = fp.split('/')[-1]

print("Scanning complete")

Scanning CSVs:   0%|          | 0/898 [00:00<?, ?it/s]

Scanning CSVs:   0%|          | 1/898 [00:00<03:04,  4.86it/s]

Scanning CSVs:   0%|          | 3/898 [00:00<01:30,  9.89it/s]

Scanning CSVs:   1%|          | 5/898 [00:00<01:22, 10.83it/s]

Scanning CSVs:   1%|          | 7/898 [00:00<01:09, 12.76it/s]

...

Scanning CSVs: 100%|█████████▉| 894/898 [00:59<00:00, 16.36it/s]

Scanning CSVs: 100%|█████████▉| 896/898 [00:59<00:00, 15.11it/s]

Scanning CSVs: 100%|██████████| 898/898 [00:59<00:00, 15.74it/s]

Scanning CSVs: 100%|██████████| 898/898 [00:59<00:00, 15.14it/s]

Scanning complete


In [7]:
# Override N=1 with optimal value (45 degrees)
manual_data = pd.DataFrame({
    'id': ['001_0'],
    'x': ['s0.0'],
    'y': ['s0.0'],
    'deg': [f's{fine_optimal_angle}']
})
xs = strip(manual_data['x'].to_numpy())
ys = strip(manual_data['y'].to_numpy())
ds = strip(manual_data['deg'].to_numpy())
sc = score_group(xs, ys, ds, tx, ty)
best[1]['score'] = float(sc)
best[1]['data'] = manual_data.copy()
best[1]['src'] = 'optimal_computed'

print(f"N=1 optimal score: {sc:.8f}")

N=1 optimal score: 0.66125000


In [8]:
# Calculate total ensemble score
total_score = sum(best[n]['score'] for n in range(1, 201))
print(f"\nEnsemble total score: {total_score:.6f}")
print(f"Baseline score: 70.676102")
print(f"Improvement: {70.676102 - total_score:.6f}")
print(f"Target: 68.919154")
print(f"Gap to target: {total_score - 68.919154:.6f}")


Ensemble total score: 27.478179
Baseline score: 70.676102
Improvement: 43.197923
Target: 68.919154
Gap to target: -41.440975


In [9]:
# Show top improvements from ensemble
baseline_df = pd.read_csv('/home/code/experiments/001_baseline/submission.csv')
baseline_df['N'] = baseline_df['id'].astype(str).str.split('_').str[0].astype(int)

improvements = []
for n in range(1, 201):
    g = baseline_df[baseline_df['N'] == n]
    if len(g) != n:
        continue
    xs = strip(g['x'].to_numpy())
    ys = strip(g['y'].to_numpy())
    ds = strip(g['deg'].to_numpy())
    baseline_score = score_group(xs, ys, ds, tx, ty)
    ensemble_score = best[n]['score']
    improvement = baseline_score - ensemble_score
    if improvement > 1e-9:
        improvements.append({
            'n': n,
            'baseline': baseline_score,
            'ensemble': ensemble_score,
            'improvement': improvement,
            'source': best[n]['src']
        })

improvements_df = pd.DataFrame(improvements)
if len(improvements_df) > 0:
    print(f"\nFound {len(improvements_df)} N values with improvements:")
    print(improvements_df.sort_values('improvement', ascending=False).head(20))
else:
    print("No improvements found - baseline is already the best ensemble")


Found 199 N values with improvements:
     n  baseline  ensemble  improvement        source
22  24  0.365506  0.017137     0.348369  ensemble.csv
39  41  0.367868  0.030209     0.337659  ensemble.csv
29  31  0.370329  0.034150     0.336179  ensemble.csv
31  33  0.369358  0.033359     0.335999  ensemble.csv
44  46  0.364452  0.029102     0.335351  ensemble.csv
42  44  0.366293  0.031419     0.334875  ensemble.csv
47  49  0.363432  0.030764     0.332668  ensemble.csv
36  38  0.366522  0.035419     0.331103  ensemble.csv
43  45  0.363523  0.033572     0.329951  ensemble.csv
40  42  0.366839  0.037907     0.328932  ensemble.csv
33  35  0.366855  0.039817     0.327038  ensemble.csv
48  50  0.360753  0.034288     0.326466  ensemble.csv
46  48  0.355530  0.029634     0.325896  ensemble.csv
45  47  0.357493  0.032102     0.325391  ensemble.csv
32  34  0.369041  0.043820     0.325220  ensemble.csv
41  43  0.370040  0.045015     0.325025  ensemble.csv
38  40  0.362148  0.037174     0.324974  en

In [10]:
# Create ensemble submission
rows = []
for n in range(1, 201):
    if best[n]['data'] is not None:
        rows.append(best[n]['data'])

ensemble_df = pd.concat(rows, ignore_index=True)
ensemble_df['sn'] = ensemble_df['id'].str.split('_').str[0].astype(int)
ensemble_df['si'] = ensemble_df['id'].str.split('_').str[1].astype(int)
ensemble_df = ensemble_df.sort_values(['sn', 'si']).drop(columns=['sn', 'si'])
ensemble_df = ensemble_df[['id', 'x', 'y', 'deg']]

print(f"Ensemble submission shape: {ensemble_df.shape}")
print(f"Expected: (20100, 4)")

# Save ensemble
os.makedirs('/home/code/experiments/002_ensemble', exist_ok=True)
ensemble_df.to_csv('/home/code/experiments/002_ensemble/submission.csv', index=False)
print("\nSaved to /home/code/experiments/002_ensemble/submission.csv")

Ensemble submission shape: (20100, 4)
Expected: (20100, 4)

Saved to /home/code/experiments/002_ensemble/submission.csv


In [11]:
# Verify ensemble score
verify_df = pd.read_csv('/home/code/experiments/002_ensemble/submission.csv')
verify_df['N'] = verify_df['id'].astype(str).str.split('_').str[0].astype(int)

verify_total = 0
for n in range(1, 201):
    g = verify_df[verify_df['N'] == n]
    xs = strip(g['x'].to_numpy())
    ys = strip(g['y'].to_numpy())
    ds = strip(g['deg'].to_numpy())
    verify_total += score_group(xs, ys, ds, tx, ty)

print(f"Verified ensemble score: {verify_total:.6f}")
print(f"Expected: {total_score:.6f}")
print(f"Match: {abs(verify_total - total_score) < 1e-6}")

Verified ensemble score: 27.478179
Expected: 27.478179
Match: True


In [12]:
# Copy to submission folder
import shutil
shutil.copy('/home/code/experiments/002_ensemble/submission.csv', '/home/submission/submission.csv')
print("Copied to /home/submission/submission.csv")

# Save metrics
import json
metrics = {'cv_score': verify_total}
with open('/home/code/experiments/002_ensemble/metrics.json', 'w') as f:
    json.dump(metrics, f)
print(f"Metrics saved: {metrics}")

Copied to /home/submission/submission.csv
Metrics saved: {'cv_score': 27.478179154216637}


In [13]:
# Something is wrong - the score is too good. Let me investigate
# Check if all N values have the correct number of trees

print("Checking ensemble data integrity...")
verify_df = pd.read_csv('/home/code/experiments/002_ensemble/submission.csv')
verify_df['N'] = verify_df['id'].astype(str).str.split('_').str[0].astype(int)

issues = []
for n in range(1, 201):
    g = verify_df[verify_df['N'] == n]
    if len(g) != n:
        issues.append({'n': n, 'expected': n, 'actual': len(g)})
        
if issues:
    print(f"Found {len(issues)} issues:")
    for issue in issues[:20]:
        print(f"  N={issue['n']}: expected {issue['expected']} trees, got {issue['actual']}")
else:
    print("All N values have correct number of trees")

# Check the source distribution
sources = {}
for n in range(1, 201):
    src = best[n]['src']
    sources[src] = sources.get(src, 0) + 1
    
print("\nSource distribution:")
for src, count in sorted(sources.items(), key=lambda x: -x[1]):
    print(f"  {src}: {count} N values")

Checking ensemble data integrity...
All N values have correct number of trees

Source distribution:
  submission.csv: 145 N values
  ensemble.csv: 37 N values
  candidate_004.csv: 13 N values
  output5.csv: 2 N values
  submission_v21.csv: 2 N values
  optimal_computed: 1 N values


In [15]:
# Check for overlaps in the ensemble submission
from shapely.strtree import STRtree

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        
        initial_polygon = Polygon(TREE_VERTICES)
        rotated = affinity.rotate(initial_polygon, self.angle, origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=self.center_x, yoff=self.center_y)

def has_overlap(trees):
    if len(trees) <= 1:
        return False, None, None, None
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                intersection = poly.intersection(polygons[idx])
                if intersection.area > 1e-10:  # Significant overlap
                    return True, i, idx, intersection.area
    return False, None, None, None

def load_trees_for_n(df, n):
    prefix = f"{n:03d}_"
    rows = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in rows.iterrows():
        x = str(row['x']).replace('s', '')
        y = str(row['y']).replace('s', '')
        deg = str(row['deg']).replace('s', '')
        trees.append(ChristmasTree(x, y, deg))
    return trees

# Check a sample of N values for overlaps
print("Checking for overlaps in ensemble submission...")
overlap_issues = []
for n in [1, 5, 10, 20, 30, 40, 50, 100, 150, 200]:
    trees = load_trees_for_n(verify_df, n)
    result = has_overlap(trees)
    if result[0]:
        print(f"  N={n}: OVERLAP between trees {result[1]} and {result[2]}, area={result[3]:.2e}")
        overlap_issues.append(n)
    else:
        print(f"  N={n}: OK")

if not overlap_issues:
    print("\nNo significant overlaps detected in sampled configurations.")

Checking for overlaps in ensemble submission...
  N=1: OK
  N=5: OVERLAP between trees 0 and 2, area=1.45e-01
  N=10: OVERLAP between trees 0 and 1, area=2.46e-01
  N=20: OVERLAP between trees 0 and 10, area=2.46e-01
  N=30: OVERLAP between trees 0 and 15, area=2.46e-01
  N=40: OVERLAP between trees 0 and 11, area=2.46e-01
  N=50: OVERLAP between trees 0 and 32, area=2.46e-01
  N=100: OVERLAP between trees 0 and 88, area=1.09e-01
  N=150: OVERLAP between trees 0 and 112, area=2.23e-03
  N=200: OVERLAP between trees 0 and 102, area=3.97e-04


In [16]:
# The ensemble.csv has invalid overlapping configurations
# Let me re-scan but only accept configurations without overlaps

print("Re-scanning with overlap validation...")

best_valid = {n: {'score': 1e300, 'data': None, 'src': None} for n in range(1, 201)}

# First, let's check which source files have valid configurations
valid_sources = set()
invalid_sources = set()

for fp in tqdm(csv_files[:100], desc='Validating sources'):  # Sample first 100
    try:
        df = pd.read_csv(fp)
    except Exception:
        continue
    
    if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
        continue
    
    # Check N=10 as a quick validation
    df_copy = df.copy()
    df_copy['N'] = df_copy['id'].astype(str).str.split('_').str[0].astype(int)
    g = df_copy[df_copy['N'] == 10]
    
    if len(g) == 10:
        trees = load_trees_for_n(df_copy, 10)
        result = has_overlap(trees)
        if result[0]:
            invalid_sources.add(fp.split('/')[-1])
        else:
            valid_sources.add(fp.split('/')[-1])

print(f"\nValid sources: {len(valid_sources)}")
print(f"Invalid sources: {len(invalid_sources)}")
print(f"\nInvalid source files:")
for src in list(invalid_sources)[:10]:
    print(f"  {src}")

Re-scanning with overlap validation...


Validating sources:   0%|          | 0/100 [00:00<?, ?it/s]

Validating sources:   3%|▎         | 3/100 [00:00<00:03, 28.39it/s]

Validating sources:   6%|▌         | 6/100 [00:00<00:04, 22.31it/s]

Validating sources:   9%|▉         | 9/100 [00:00<00:04, 20.93it/s]

...

Validating sources:  91%|█████████ | 91/100 [00:03<00:00, 25.41it/s]

Validating sources:  94%|█████████▍| 94/100 [00:03<00:00, 24.28it/s]

Validating sources:  98%|█████████▊| 98/100 [00:04<00:00, 24.54it/s]

Validating sources: 100%|██████████| 100/100 [00:04<00:00, 23.80it/s]


Valid sources: 39
Invalid sources: 4

Invalid source files:
  candidate_002.csv
  submission_iter_1.csv
  submission_iter_2.csv
  submission.csv


In [18]:
# Let's use the baseline as starting point and only accept valid improvements
# The baseline is known to be valid (score 70.676102)

baseline_df = pd.read_csv('/home/code/experiments/001_baseline/submission.csv')
baseline_df['N'] = baseline_df['id'].astype(str).str.split('_').str[0].astype(int)

# Calculate baseline scores per N
baseline_scores = {}
for n in range(1, 201):
    g = baseline_df[baseline_df['N'] == n]
    xs = strip(g['x'].to_numpy())
    ys = strip(g['y'].to_numpy())
    ds = strip(g['deg'].to_numpy())
    baseline_scores[n] = score_group(xs, ys, ds, tx, ty)

print(f"Baseline total: {sum(baseline_scores.values()):.6f}")

# Now scan all CSVs and find valid improvements
best_valid = {n: {'score': baseline_scores[n], 'data': baseline_df[baseline_df['N'] == n].drop(columns=['N']).copy(), 'src': 'baseline'} for n in range(1, 201)}

improvements_found = 0
for fp in tqdm(csv_files, desc='Scanning for valid improvements'):
    try:
        df = pd.read_csv(fp)
    except Exception:
        continue
    
    if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
        continue
    
    df = df.copy()
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    
    for n, g in df.groupby('N'):
        if n < 1 or n > 200:
            continue
        if len(g) != n:
            continue
        
        try:
            xs = strip(g['x'].to_numpy())
            ys = strip(g['y'].to_numpy())
            ds = strip(g['deg'].to_numpy())
            sc = score_group(xs, ys, ds, tx, ty)
            
            # Only consider if better than current best
            if sc < best_valid[n]['score'] - 1e-9:
                # Validate no overlaps
                trees = load_trees_for_n(df, n)
                result = has_overlap(trees)
                if not result[0]:  # No overlap
                    best_valid[n]['score'] = float(sc)
                    best_valid[n]['data'] = g.drop(columns=['N']).copy()
                    best_valid[n]['src'] = fp.split('/')[-1]
                    improvements_found += 1
        except Exception as e:
            continue

print(f"\nFound {improvements_found} valid improvements")

Baseline total: 70.676102


Scanning for valid improvements:   0%|          | 0/898 [00:00<?, ?it/s]

Scanning for valid improvements:   0%|          | 2/898 [00:00<01:03, 14.03it/s]

Scanning for valid improvements:   0%|          | 4/898 [00:00<01:00, 14.88it/s]

Scanning for valid improvements:   1%|          | 6/898 [00:00<01:11, 12.55it/s]

...

Scanning for valid improvements: 100%|█████████▉| 894/898 [01:34<00:00, 12.80it/s]

Scanning for valid improvements: 100%|█████████▉| 896/898 [01:34<00:00, 13.94it/s]

Scanning for valid improvements: 100%|██████████| 898/898 [01:34<00:00, 12.77it/s]

Scanning for valid improvements: 100%|██████████| 898/898 [01:34<00:00,  9.51it/s]


Found 391 valid improvements


In [19]:
# Calculate the new valid ensemble score
valid_total = sum(best_valid[n]['score'] for n in range(1, 201))
print(f"Valid ensemble total score: {valid_total:.6f}")
print(f"Baseline score: 70.676102")
print(f"Improvement: {70.676102 - valid_total:.6f}")
print(f"Target: 68.919154")
print(f"Gap to target: {valid_total - 68.919154:.6f}")

# Show source distribution
sources = {}
for n in range(1, 201):
    src = best_valid[n]['src']
    sources[src] = sources.get(src, 0) + 1
    
print("\nSource distribution:")
for src, count in sorted(sources.items(), key=lambda x: -x[1])[:15]:
    print(f"  {src}: {count} N values")

Valid ensemble total score: 70.646824
Baseline score: 70.676102
Improvement: 0.029278
Target: 68.919154
Gap to target: 1.727670

Source distribution:
  submission.csv: 111 N values
  baseline: 45 N values
  santa-2025.csv: 42 N values
  ensemble_valid.csv: 2 N values


In [20]:
# Create valid ensemble submission
rows = []
for n in range(1, 201):
    if best_valid[n]['data'] is not None:
        rows.append(best_valid[n]['data'])

valid_ensemble_df = pd.concat(rows, ignore_index=True)
valid_ensemble_df['sn'] = valid_ensemble_df['id'].str.split('_').str[0].astype(int)
valid_ensemble_df['si'] = valid_ensemble_df['id'].str.split('_').str[1].astype(int)
valid_ensemble_df = valid_ensemble_df.sort_values(['sn', 'si']).drop(columns=['sn', 'si'])
valid_ensemble_df = valid_ensemble_df[['id', 'x', 'y', 'deg']]

print(f"Valid ensemble submission shape: {valid_ensemble_df.shape}")

# Save valid ensemble
os.makedirs('/home/code/experiments/002_valid_ensemble', exist_ok=True)
valid_ensemble_df.to_csv('/home/code/experiments/002_valid_ensemble/submission.csv', index=False)
print("Saved to /home/code/experiments/002_valid_ensemble/submission.csv")

# Copy to submission folder
shutil.copy('/home/code/experiments/002_valid_ensemble/submission.csv', '/home/submission/submission.csv')
print("Copied to /home/submission/submission.csv")

# Save metrics
metrics = {'cv_score': valid_total}
with open('/home/code/experiments/002_valid_ensemble/metrics.json', 'w') as f:
    json.dump(metrics, f)
print(f"Metrics saved: {metrics}")

Valid ensemble submission shape: (20100, 4)
Saved to /home/code/experiments/002_valid_ensemble/submission.csv
Copied to /home/submission/submission.csv
Metrics saved: {'cv_score': 70.64682353047581}
